<a href="https://colab.research.google.com/github/AlexBorealis/netology_recsys_hw/blob/master/kozlov_hw_3_recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise LightFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for LightFM: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831160 sha256=768fc789dc944763bc826e8bf5ee9c9ddcd0a1b98aa25fe317e43a3d4cab1c18
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505214 sha256=75ebb9f0643ad3129a1d5183eece44bbe316fd68d5796b86182d77a78cf437cb
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built LightFM scikit-surprise


In [2]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2025-04-13 13:48:05--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 173.194.79.138, 173.194.79.113, 173.194.79.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.79.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2025-04-13 13:48:05--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.18.132, 2a00:1450:4013:c18::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.18.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.008s  

2025-04-13 13:48:08 (106 MB/s) - 

In [4]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [16]:
# Load the movielens-100K dataset
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# **Renaming movies index**

In [17]:
movies.movieId = movies.index

# **Cleaning data**

In [18]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop= True)
movies_with_ratings.dropna(inplace= True)
print(movies_with_ratings.shape)
movies_with_ratings.head()

(77939, 6)


,movieId,title,genres,userId,rating,timestamp
0,1,Jumanji (1995),Adventure|Children|Fantasy,1,4.0,964982703
1,1,Jumanji (1995),Adventure|Children|Fantasy,5,4.0,847434962
2,1,Jumanji (1995),Adventure|Children|Fantasy,7,4.5,1106635946
3,1,Jumanji (1995),Adventure|Children|Fantasy,15,2.5,1510577970
4,1,Jumanji (1995),Adventure|Children|Fantasy,17,4.5,1305696483


# **Transforming data to wide format**

In [19]:
interactions = movies_with_ratings.groupby(['userId', 'title'])['rating'].count().unstack().fillna(0)
print(interactions.shape)
interactions.head()

(610, 5389)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...All the Marbles (1981),...And Justice for All (1979),1-900 (06) (1994),...,Zone 39 (1997),"Zone, The (La Zona) (2007)",Zoolander (2001),Zoom (2006),[REC] (2007),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Transforming wide data to csr matrix**

In [20]:
interaction_sparse = sparse.csr_matrix(interactions.values)
interaction_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 77939 stored elements and shape (610, 5389)>

## **content based**

In [21]:
def change_string(str_):
    return str(str_).replace('|', ' ').lower()

movies_with_ratings['norm_genres'] = movies_with_ratings['genres'].apply(change_string)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp,norm_genres
0,1,Jumanji (1995),Adventure|Children|Fantasy,1,4.0,964982703,adventure children fantasy
1,1,Jumanji (1995),Adventure|Children|Fantasy,5,4.0,847434962,adventure children fantasy
2,1,Jumanji (1995),Adventure|Children|Fantasy,7,4.5,1106635946,adventure children fantasy
3,1,Jumanji (1995),Adventure|Children|Fantasy,15,2.5,1510577970,adventure children fantasy
4,1,Jumanji (1995),Adventure|Children|Fantasy,17,4.5,1305696483,adventure children fantasy


In [22]:
movies_with_ratings.tail()

,movieId,title,genres,userId,rating,timestamp,norm_genres
77934,9010,The Runner (2015),Drama,474,3.0,1117481084,drama
77935,9010,The Runner (2015),Drama,599,3.0,1498523557,drama
77936,9018,"Visit, The (2015)",Comedy|Horror,221,5.0,1121365559,comedy horror
77937,9018,"Visit, The (2015)",Comedy|Horror,462,4.0,1123891722,comedy horror
77938,9018,"Visit, The (2015)",Comedy|Horror,474,4.0,1101737052,comedy horror


# **Tf-idf transormation**

In [23]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_ratings['norm_genres'])
features = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns= features)

In [24]:
tfidf_df['movieId'] = movies_with_ratings['movieId']
print(tfidf_df.shape)
tfidf_df.head()

(77939, 25)


,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,musical,mystery,no,noir,romance,sci,thriller,war,western,movieId
0,0.0,0.525383,0.0,0.597637,0.0,0.0,0.0,0.0,0.605642,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.525383,0.0,0.597637,0.0,0.0,0.0,0.0,0.605642,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.525383,0.0,0.597637,0.0,0.0,0.0,0.0,0.605642,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.525383,0.0,0.597637,0.0,0.0,0.0,0.0,0.605642,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.525383,0.0,0.597637,0.0,0.0,0.0,0.0,0.605642,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## **interaction based**

In [25]:
ds = Dataset()
ds.fit(
    users= movies_with_ratings['userId'].unique(),
    items= movies_with_ratings['movieId'],
    item_features= features
    )

In [26]:
data = []
for _, rows in movies_with_ratings.iterrows():
    data.append((rows.userId, rows.movieId))

In [27]:
interaction_matrix = ds.build_interactions(data)[0]
interaction_matrix

<COOrdinate sparse matrix of dtype 'int32'
	with 77939 stored elements and shape (610, 5390)>

In [28]:
def transform_features(features, id_name):
    """
    Returns iterable of (id, [list of feature names that has value])
    """
    transformed_features = []
    for row in features.to_dict(orient= 'records'):
        id_value = row[id_name]
        del row[id_name]
        feature_names = {key: value for key, value in row.items() if value != 0}
        transformed_features.append((id_value, feature_names))

    return transformed_features

In [29]:
item_features_matrix = ds.build_item_features(transform_features(tfidf_df, 'movieId'))
item_features_matrix

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 18113 stored elements and shape (5390, 5414)>

# **LightFM**

In [30]:
model = LightFM(no_components= 50, loss= 'warp', random_state= 42)
model.fit(
    interaction_matrix,
    epochs= 30,
    num_threads= 3,
    item_features= item_features_matrix
    )

# **Creation recommendation**

In [49]:
n_items = interactions.shape[1]

user_id = 2
seen = np.where(interactions.loc[user_id] == 1)

all_titles = np.arange(n_items)
new_titles = np.delete(all_titles, seen)

recommendations = pd.DataFrame({
    'title': movies.title[new_titles],
    'score': model.predict(user_id, new_titles)
}).sort_values(ascending= False, by= 'score')

recommendations.iloc[np.random.choice(np.arange(0, 20), size= 10, replace= False)].sort_values(ascending= False, by= 'score')

,title,score
510,"Silence of the Lambs, The (1991)",93.223740
224,Star Wars: Episode IV - A New Hope (1977),85.681526
4131,Maid in Manhattan (2002),82.838310
2224,Home Alone 2: Lost in New York (1992),82.233345
897,Cheech and Chong's Up in Smoke (1978),81.491859
97,Braveheart (1995),79.101372
334,Speed (1994),73.417236
1938,"Walk on the Moon, A (1999)",70.873230
257,Pulp Fiction (1994),68.393005
0,Toy Story (1995),67.359795


In [50]:
movies_with_ratings.loc[movies_with_ratings['userId'] == 2].sort_values(ascending= False, by= 'rating')

,movieId,title,genres,userId,rating,timestamp,norm_genres
34762,1704,Edward Scissorhands (1990),Drama|Fantasy|Romance,2,4.5,1445715228,drama fantasy romance
9186,333,"River Wild, The (1994)",Action|Thriller,2,4.0,1445715029,action thriller
72615,6874,Pride and Glory (2008),Crime|Drama,2,4.0,1445714952,crime drama
55517,3578,Spy Game (2001),Action|Crime|Drama|Thriller,2,4.0,1445714885,action crime drama thriller
76949,8798,The Cobbler (2015),Comedy|Drama|Fantasy,2,3.5,1445714960,comedy drama fantasy
8652,318,I Love Trouble (1994),Action|Comedy,2,3.0,1445714835,action comedy


In [52]:
recommendations.head(20)

,title,score
314,Forrest Gump (1994),99.236366
510,"Silence of the Lambs, The (1991)",93.223740
224,Star Wars: Episode IV - A New Hope (1977),85.681526
4131,Maid in Manhattan (2002),82.838310
2224,Home Alone 2: Lost in New York (1992),82.233345
897,Cheech and Chong's Up in Smoke (1978),81.491859
97,Braveheart (1995),79.101372
507,Terminator 2: Judgment Day (1991),77.742035
277,"Shawshank Redemption, The (1994)",76.125793
506,Aladdin (1992),75.448296
